# Neural Network for all lipids

### Import

In [134]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.initializers import he_normal
from sklearn.metrics import mean_squared_error, r2_score
from keras.regularizers import l2
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### Reproducibility

In [135]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Data loading

In [136]:
# Dataset path 
dataset_dir = 'data/processed_data'
# Training loading
training_input_path = os.path.join(dataset_dir, 'train_features.parquet')
training_output_path = os.path.join(dataset_dir, 'train_targets.parquet')
# Testing loading
testing_input_path = os.path.join(dataset_dir, 'test_features.parquet')
testing_output_path = os.path.join(dataset_dir, 'test_targets.parquet')

In [137]:
# Load dataset into Pandas dataframes
training_input = pd.read_parquet(training_input_path)
training_output = pd.read_parquet(training_output_path)

testing_input = pd.read_parquet(testing_input_path)
testing_output = pd.read_parquet(testing_output_path)

lipids_to_drop = ['LPC O-18:3', 'Cer 36:1', 'PE(30:1) ', 'PE 34:2', 'PA(P-38:6)\xa0',
 'PI-Cer(t30:2)\xa0', 'PE 36:3', 'PE O-39:6', 'PA 40:6', 'PA 42:4',
 'PE(O-40:6)\xa0', 'PE(42:6)\xa0', 'PE(40:8)\xa0', 'PGP(34:1) ', 'PE(44:11(OH))\xa0',
 'PS(40:4) ', 'PIP(O-36:5)\xa0', 'PI-Cer(t30:0)\xa0']

training_output = training_output.drop(lipids_to_drop, axis=1)

# Number of output nodes (lipids) for the model
OUTPUT_NODES = training_output.shape[1]
# Number of input nodes (genes) for the model
input_dim = training_input.shape[1]
# Batch size
batch_size = 32

In [138]:
def lr_schedule(epoch):
    initial_learning_rate = 0.1  # Set your initial learning rate here
    decay_factor = 0.9  # Set your decay factor here
    lr = initial_learning_rate * decay_factor ** epoch
    return lr

# Create a Sequential model
def build_model(summary=False):
    model = Sequential()

    # Add the input layer with 500 nodes
    model.add(Dense(128, input_dim=500, kernel_initializer=he_normal(seed=seed), activation='gelu'))
    model.add(Dense(128, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(64, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(64, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(32, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(32, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(16, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(16, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(8, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(8, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    # Add the output layer with OUTPUT_NODES nodes (for multiple regression)
    model.add(Dense(OUTPUT_NODES, activation='linear'))

    if summary:
        # Display the model summary
        model.summary()

    return model

# Print the model summary
build_model(True)

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_524 (Dense)           (None, 128)               64128     
                                                                 
 dense_525 (Dense)           (None, 128)               16512     
                                                                 
 dense_526 (Dense)           (None, 64)                8256      
                                                                 
 dense_527 (Dense)           (None, 64)                4160      
                                                                 
 dense_528 (Dense)           (None, 32)                2080      
                                                                 
 dense_529 (Dense)           (None, 32)                1056      
                                                                 
 dense_530 (Dense)           (None, 16)              

In [139]:
# Calculate loss for each output node (lipid)
def calculate_losses(lipid_true: pd.DataFrame, lipid_pred: np.ndarray):
    losses = np.zeros((OUTPUT_NODES,))
    for i in range(OUTPUT_NODES):
        y_pred = lipid_pred[:, i]
        y_true = lipid_true.iloc[:, i]
        losses[i] = mean_squared_error(y_true, y_pred)
    return losses

# Calculate loss for each output node (lipid)
def calculate_r2s(lipid_true: pd.DataFrame, lipid_pred: np.ndarray):
    r2s = np.zeros((OUTPUT_NODES,))
    for i in range(OUTPUT_NODES):
        y_pred = lipid_pred[:, i]
        y_true = lipid_true.iloc[:, i]
        r2s[i] = r2_score(y_true, y_pred)
    return r2s

In [140]:
# Use early stopping on the validation
early_stopping = EarlyStopping(monitor='val_loss',  # Metric chose is validation loss (MSE)
                               patience=10,         # Number of epochs with no improvement after which training stops
                               restore_best_weights=True)  # Restores model weights from the epoch with the best value of the monitored metric

In [142]:
lipid_names = list(map(lambda s: s.strip(), training_output.columns.values))
lipids_metrics_avg = pd.DataFrame(columns=['Loss', 'R2', 'R2_no_scale'], index=lipid_names)

print(f'Start training')
num_folds = 5
# Define the K-fold Cross Validator
k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# Metrics for each fold
loss_per_fold = np.zeros((num_folds,len(lipid_names)))
r2_per_fold = np.zeros((num_folds,len(lipid_names)))
r2_per_fold_no_scale = np.zeros((num_folds,len(lipid_names)))

# K-fold Cross Validation model evaluation
fold_no = 1
models = []
split_indices = []
for input_indices, output_indices in k_fold.split(training_input, training_output):
    split_indices.append(output_indices)
    # Build the model
    model = build_model()

    # Fit and transform the data
    scaler = MinMaxScaler()
    scaler.fit(training_output.iloc[input_indices])
    training_output_scale = pd.DataFrame(scaler.transform(training_output.iloc[input_indices]), columns=training_output.columns, index=training_output.iloc[input_indices].index)
    validation_output_scale = pd.DataFrame(scaler.transform(training_output.iloc[output_indices]), columns=training_output.columns, index=training_output.iloc[output_indices].index)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error',
                  metrics=[metrics.mean_squared_error, metrics.mean_absolute_error, tfa.metrics.RSquare(), metrics.mean_absolute_percentage_error])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Train the model
    history = model.fit(
        training_input.iloc[input_indices], training_output_scale,
        epochs=200,  # Adjust the number of epochs as needed
        batch_size = 32,
        validation_data=(training_input.iloc[output_indices], validation_output_scale),
        callbacks=[early_stopping],
        shuffle=True
    ).history

    # Generate generalization metrics
    prediction = model.predict(training_input.iloc[output_indices])
    losses = calculate_losses(validation_output_scale, prediction)
    r2s = calculate_r2s(validation_output_scale, prediction)
    loss_per_fold[fold_no-1] = losses
    r2_per_fold[fold_no-1] = r2s
    reversed_data = scaler.inverse_transform(prediction)
    r2_per_fold_no_scale[fold_no-1] = calculate_r2s(training_output.iloc[output_indices], reversed_data)

    # Increase fold number
    fold_no = fold_no + 1

mean_loss = loss_per_fold.mean(axis=0)
mean_r2 = r2_per_fold.mean(axis=0)
mean_r2_no_scale = r2_per_fold_no_scale.mean(axis=0)
lipids_metrics_avg['Loss'] = mean_loss
lipids_metrics_avg['R2'] = mean_r2
lipids_metrics_avg['R2_no_scale'] = mean_r2_no_scale
print('#'*72)
print(f'Finish training')
print(lipids_metrics_avg)

lipids_metrics_avg.to_csv('results/lipids_metrics_avg_neural_network.csv')

Start training


/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
1565/1565 [==============================] - 26s 13ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - mean_absolute_error: 0.0642 - r_square: 0.0962 - mean_absolute_percentage_error: 8465134.0000 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_mean_absolute_error: 0.0573 - val_r_square: 0.2643 - val_mean_absolute_percentage_error: 7399490.0000
Epoch 2/200
1565/1565 [==============================] - 17s 11ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - mean_absolute_error: 0.0553 - r_square: 0.3095 - mean_absolute_percentage_error: 7143448.0000 - val_loss: 0.0062 - val_mean_squared_error: 0.0062 - val_mean_absolute_error: 0.0544 - val_r_square: 0.3112 - val_mean_absolute_percentage_error: 6867838.0000
Epoch 3/200
1565/1565 [==============================] - 18s 11ms/step - loss: 0.0057 - mean_squared_error: 0.0057 - mean_absolute_error: 0.0523 - r_square: 0.36

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/200
1565/1565 [==============================] - 22s 11ms/step - loss: 0.0082 - mean_squared_error: 0.0082 - mean_absolute_error: 0.0630 - r_square: 0.1242 - mean_absolute_percentage_error: 8222273.0000 - val_loss: 0.0065 - val_mean_squared_error: 0.0065 - val_mean_absolute_error: 0.0561 - val_r_square: 0.2997 - val_mean_absolute_percentage_error: 7160571.0000
Epoch 2/200
1565/1565 [==============================] - 15s 10ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - mean_absolute_error: 0.0539 - r_square: 0.3366 - mean_absolute_percentage_error: 6893206.0000 - val_loss: 0.0060 - val_mean_squared_error: 0.0060 - val_mean_absolute_error: 0.0541 - val_r_square: 0.3455 - val_mean_absolute_percentage_error: 6615216.0000
Epoch 3/200
1565/1565 [==============================] - 16s 10ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - mean_absolute_error: 0.0517 - r_square: 0.37

...

------------------------------------------------------------------------
Training for fold 4 ...
Epoch 1/200
1565/1565 [==============================] - 15s 7ms/step - loss: 0.0085 - mean_squared_error: 0.0085 - mean_absolute_error: 0.0636 - r_square: 0.0967 - mean_absolute_percentage_error: 8335057.5000 - val_loss: 0.0063 - val_mean_squared_error: 0.0063 - val_mean_absolute_error: 0.0558 - val_r_square: 0.2921 - val_mean_absolute_percentage_error: 7432479.0000
Epoch 2/200
1565/1565 [==============================] - 13s 8ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - mean_absolute_error: 0.0539 - r_square: 0.3239 - mean_absolute_percentage_error: 6934054.0000 - val_loss: 0.0056 - val_mean_squared_error: 0.0056 - val_mean_absolute_error: 0.0518 - val_r_square: 0.3585 - val_mean_absolute_percentage_error: 6192133.0000
Epoch 3/200
1565/1565 [==============================] - 11s 7ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - mean_absolute_error: 0.0517 - r_square: 0.3665 

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/200
1565/1565 [==============================] - 13s 6ms/step - loss: 0.0082 - mean_squared_error: 0.0082 - mean_absolute_error: 0.0626 - r_square: 0.1053 - mean_absolute_percentage_error: 8230913.0000 - val_loss: 0.0062 - val_mean_squared_error: 0.0062 - val_mean_absolute_error: 0.0552 - val_r_square: 0.2929 - val_mean_absolute_percentage_error: 7530947.0000
Epoch 2/200
1565/1565 [==============================] - 7s 5ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - mean_absolute_error: 0.0530 - r_square: 0.3315 - mean_absolute_percentage_error: 6838280.0000 - val_loss: 0.0055 - val_mean_squared_error: 0.0055 - val_mean_absolute_error: 0.0513 - val_r_square: 0.3621 - val_mean_absolute_percentage_error: 6317762.0000
Epoch 3/200
1565/1565 [==============================] - 9s 6ms/step - loss: 0.0055 - mean_squared_error: 0.0055 - mean_absolute_error: 0.0510 - r_square: 0.3706 - 

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

########################################################################
Finish training
                 Loss        R2  R2_no_scale
LPC O- 18:3  0.008279  0.248020     0.248020
LPC 15:1     0.003408  0.770890     0.770890
LPC 20:4     0.003939  0.434616     0.434616
LPC 22:6     0.006434  0.427899     0.427899
Cer 40:2     0.002369  0.606437     0.606437
...               ...       ...          ...
LPC O-16:2   0.007378  0.590229     0.590229
LPC 16:0     0.008747  0.327499     0.327499
LPC O-18:2   0.011308  0.218277     0.218277
LPC 18:0     0.003151  0.298383     0.298383
PC 36:4      0.003140  0.801568     0.801568

[138 rows x 3 columns]


In [143]:
# Print k best and worst lipids for average between folds of metrics loss and r2
k = 5

def format_names(df: pd.DataFrame, just=15):
    return ', '.join([name.rjust(just) for name in df.index.values])

def format_values(df: pd.DataFrame, col: str, just=15):
    return ', '.join([f'{val:.5e}'.rjust(just) for val in df[col].values])

best_losses = lipids_metrics_avg.nsmallest(k, 'Loss')
worst_losses = lipids_metrics_avg.nlargest(k, 'Loss')
print("Loss:")
print("  Best:")
print(f"  {format_names(best_losses)}")
print(f"  {format_values(best_losses, 'Loss')}")
print("  Worst:")
print(f"  {format_names(worst_losses)}")
print(f"  {format_values(worst_losses, 'Loss')}")

best_r2s = lipids_metrics_avg.nlargest(k, 'R2')
worst_r2s = lipids_metrics_avg.nsmallest(k, 'R2')
print("R2:")
print("  Best:")
print(f"  {format_names(best_r2s)}")
print(f"  {format_values(best_r2s, 'R2')}")
print("  Worst:")
print(f"  {format_names(worst_r2s)}")
print(f"  {format_values(worst_r2s, 'R2')}")

Loss:
  Best:
          PE 37:2,     HexCer 36:0,         PS 36:1,         SM 42:2,         PC 36:2
      7.49796e-04,     7.98043e-04,     8.04225e-04,     8.52747e-04,     8.84003e-04
  Worst:
          PC 35:0,   PE-Cer(d38:1),        PG(40:3),    PE(40:2(OH)),   PI-Cer(t28:0)
      1.59181e-02,     1.46831e-02,     1.42634e-02,     1.27249e-02,     1.24000e-02
R2:
  Best:
         PA(34:1),        PG(34:0),         PC 36:4,        PG(38:4),         PC 32:0
      8.07043e-01,     8.04396e-01,     8.01568e-01,     7.91559e-01,     7.87372e-01
  Worst:
         PE(36:2),       PE O-38:5,       PE O-34:2,         PC 33:1,         PC 34:2
      6.35354e-02,     7.25592e-02,     9.57352e-02,     1.00466e-01,     1.15265e-01
